In [1]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image
from nim_env import NimEnv, OptimalPlayer

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


env = NimEnv(seed = 3)

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Discover the environment


In [2]:
env.__dict__

{'n_heap': 3,
 'n_agents': 2,
 'current_player': 0,
 'winner': None,
 'end': False,
 'num_step': 0,
 'heaps': [2, 5, 6],
 'heap_avail': [True, True, True],
 'heap_keys': ['1', '2', '3']}

In [3]:
env.render()
env.step([1,2])
env.render()

───────────────────────────────────
Heap 1: ||              	 (2)
───────────────────────────────────
Heap 2: |||||           	 (5)
───────────────────────────────────
Heap 3: ||||||          	 (6)
───────────────────────────────────
───────────────────────────────────
Heap 1:                 	 (0)
───────────────────────────────────
Heap 2: |||||           	 (5)
───────────────────────────────────
Heap 3: ||||||          	 (6)
───────────────────────────────────


In [6]:
env.observe()

([0, 5, 6], False, None)

# First test of DQN

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [3]:
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(9, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 21)

    def forward(self, x):
        x = (x.to(device))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [4]:
GAMMA = 0.99
buffer_size = 10000
BATCH_SIZE = 64


In [5]:
def to_input(heaps):
    #change the format of the heaps so that it can be used as an input for the neural network. 
    init_state = torch.zeros(9)
    for i in range(3):
        state = bin(heaps[i])[2:]
        j = 0 
        while j < len(state):
            init_state[i*3 + 2 - j] = np.int16(state[len(state) - 1 - j])
            j += 1
    return init_state.clone().detach()


In [6]:
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 500

heaps, _, __ = env.observe()
inite_state = to_input(heaps)

policy_net = DQN().to(device)
target_net = DQN().to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr = 5*1e-4)
memory = ReplayMemory(buffer_size)

steps_done = 0


def select_action(state):
    #will select an action accordingly to an epsilon greedy policy. 
    # Simply put, we’ll sometimes use our model for choosing the action, and sometimes we’ll just sample one uniformly. 
    # The probability of choosing a random action will start at EPS_START and will decay exponentially towards EPS_END. 
    # EPS_DECAY controls the rate of the decay.
    # returns an array of size 9.
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            #print("state shape : ", state.shape)
            #print(state)
            q = policy_net(state)
            argmax = torch.argmax(q)
            result = torch.tensor([argmax.div(7, rounding_mode="floor")+1, torch.remainder(argmax+1, 7)])
            #print("select action :", result)
            return result
    else:
        return torch.tensor([random.randrange(1,3), random.randrange(1,7)], device=device, dtype=torch.long)


episode_durations = []

def plot_durations():
    # a helper for plotting the durations of episodes, along with an average over the last 100 episodes (the measure used in the official evaluations). 
    # The plot will be underneath the cell containing the main training loop, and will update after every episode.
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())


In [7]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))
    #print("batch reward: ", batch.reward)

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    #print("non final mask : ", non_final_mask.shape)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    #print("state batch shape : ", torch.t(state_batch.view(9,-1)).shape)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    #print("action batch shape : ", action_batch.shape)
    q = (policy_net(torch.t(state_batch.view(9,-1)))) #q values given by the model
    """
    print(" q shape : ", q.shape)
    argmax = torch.argmax(q, dim = 1) #take the biggest q value
    print(argmax)
    new_index_1 = torch.div(argmax+1, 7, rounding_mode="floor") 
    new_index_2 = torch.remainder(argmax+1, 7)
    print("new indices: ")
    print(new_index_1)
    print(new_index_2)
    state_action_values = torch.zeros(len(action_batch), device = device)
    state_action_values[::2] = new_index_1
    state_action_values[1:][::2] = new_index_2
    print(state_action_values)"""
    state_action_values = q
    #print("state action values shape : ", state_action_values.shape)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    
    q_V = target_net(torch.t(non_final_next_states.view(9,-1)))
    """
    new_index_1_V = torch.div(argmax_V+1, 7, rounding_mode="floor") 
    new_index_2_V = torch.remainder(argmax_V+1, 7)
    print("New indices 1 v", new_index_1_V.shape)
    print("New indices 2 v", new_index_2_V.shape)"""
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    max_q_V = q_V.max(1)[0].detach()
    #print("max q V :" , max_q_V)
    #print("has shape ", max_q_V.shape)
    #print("must have shape ",next_state_values[non_final_mask].shape)
    next_state_values[non_final_mask] = max_q_V

    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    #print(expected_state_action_values.shape)
    
    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [23]:
def optimize_model(): #new version
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))
    #print("batch reward: ", batch.reward)

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    #print("non final mask : ", non_final_mask.shape)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    #print("state batch shape : ", torch.t(state_batch.view(9,-1)).shape)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    print("reward batch : ", reward_batch.shape)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    #print("action batch shape : ", action_batch.view(-1, 2).shape)
    q = (policy_net(torch.t(state_batch.view(9,-1)))) #q values given by the model
    #print("q : ", q.shape)
    state_action_values = q
    #print("state action values shape : ", state_action_values.shape)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    
    q_V = target_net(torch.t(non_final_next_states.view(9,-1)))
    next_state_values = torch.zeros((BATCH_SIZE, 21), device=device)
    max_q_V, argmax = q_V.max(1)
    print(argmax)
    #print("max q V :" , max_q_V)
    #print("has shape ", max_q_V.shape)
    #print("must have shape ",next_state_values[non_final_mask].shape)
    next_state_values[non_final_mask, argmax] = max_q_V.detach()

    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    print("expected : ", expected_state_action_values.unsqueeze(1).shape)
    
    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [24]:
num_episodes = 50
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    heaps, _, __ = env.observe()
    state = to_input(heaps)
    for t in count():
        # Select and perform an action
        #print("state : ", state)
        action = select_action(state)
        is_available = env.check_valid(action)
        #print("action :", action)
        if not is_available: 
            #i.e. if the action is not valid
            #print("is not available : ")
            reward = torch.tensor([-1], device=device)
            next_state = None
            memory.push(state, action, next_state, reward)
            break

        else: 
            #if the action is valid, we make a step:
            _, reward, done = env.step(action)
            reward = torch.tensor([reward], device=device)
            
            if not done:
                # Observe new state
                heaps, _, __ = env.observe()
                next_state = to_input(heaps)
            else:
                next_state = None

            # Store the transition in memory
            memory.push(state, action, next_state, reward)

            # Move to the next state
            state = next_state

            # Perform one step of the optimization (on the policy network)
            optimize_model()
            if done:
                episode_durations.append(t + 1)
                plot_durations()
                break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
plt.ioff()
plt.show()

reward batch :  torch.Size([64])
tensor([18, 18, 18, 18, 13, 18, 18, 18, 13, 18, 18, 18, 18, 18, 13, 18, 18, 18,
        18, 18, 18, 13, 18, 18, 13, 13, 18, 18, 18])


RuntimeError: The size of tensor a (21) must match the size of tensor b (64) at non-singleton dimension 1

# Play against one player

In [20]:
Turns = np.array([0,1])
for i in range(5):
    env.reset()
    heaps, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt_1 = OptimalPlayer(epsilon=0., player=Turns[0]) #optimal player
    player_opt_2 = OptimalPlayer(epsilon=0., player=Turns[1]) #our player
    while not env.end:
        if env.current_player == player_opt_1.player:
            move = player_opt_1.act(heaps)
        else:
            state = to_input(heaps)
            move = select_action(state)

        heaps, end, winner = env.step(move)

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Optimal player 1 = ' +  str(Turns[0]))
            print('Optimal player 2 = ' +  str(Turns[1]))
            env.reset()
            break

AssertionError: The selected heap is already empty